In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
path = r'./data_to_merge'

In [50]:
os.listdir(r'./data_to_merge')

['tr_joined_parsed',
 'geocoder_results_new.frt',
 '.DS_Store',
 'ks_joined_parsed',
 'ipc_preprocessed.frt',
 'geocoder_address_weather_results.frt',
 'kr_joined_parsed']

In [576]:
geocoder = pd.read_feather(path + '/geocoder_results_final.frt')
weather = pd.read_feather(path + '/weather_results_final.frt')
kr = pd.read_feather(path + r'/kr_joined_parsed (1).')
ks = pd.read_feather(path + r'/ks_joined_parsed (1).')
tr = pd.read_feather(path + r'/tr_joined_parsed (1).')
ipc = pd.read_feather(path + r'/ipc_preprocessed.frt')

In [577]:
data_lists = pd.read_feather(path + r'/data_lists_on.feather')

In [578]:
# geocoder = geocoder.merge(data_lists, left_on='raw_string', right_on='clean_address')
data_lists=data_lists.merge(geocoder[['raw_string', 'geocoder_address']], right_on='raw_string', left_on='clean_address', how='left')

In [579]:
kr = kr.merge(geocoder, left_on='clean_address', right_on='raw_string')
ks = ks.merge(geocoder, left_on='clean_address', right_on='raw_string')
tr = tr.merge(geocoder, left_on='clean_address', right_on='raw_string')

In [580]:
kr=kr.drop(columns=['raw_string_x', 'raw_string_y', 'clean_address'])
ks=ks.drop(columns=['raw_string_x', 'raw_string_y', 'clean_address'])
tr=tr.drop(columns=['raw_string_x', 'raw_string_y', 'clean_address'])

In [581]:
kr=kr.rename(columns={'amount_money':'amount_money_kr'})
ks=ks.rename(columns={'amount_money':'amount_money_ks'})
tr=tr.rename(columns={'amount_money':'amount_money_tr'})

In [582]:
exp = pd.read_feather(path + r'/target.frt')

In [583]:
exp = exp.rename(columns={'date':'month'})

In [584]:
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6264 entries, 0 to 6263
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   raw_address       6264 non-null   object        
 1   month             6264 non-null   datetime64[ns]
 2   paid              6264 non-null   float64       
 3   geocoder_address  6264 non-null   object        
 4   geocoder_lat      6264 non-null   float64       
 5   geocoder_lon      6264 non-null   float64       
 6   area              6264 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 342.7+ KB


In [585]:
kr['key_merge'] = kr['geocoder_address'] + kr['month'].astype(str)
ks['key_merge'] = ks['geocoder_address'] + ks['month'].astype(str)
tr['key_merge'] = tr['geocoder_address'] + tr['month'].astype(str)
exp['key_merge'] = exp['geocoder_address'] + exp['month'].astype(str)

In [586]:
kr = kr.drop_duplicates(subset=['key_merge']).reset_index(drop=True)
ks = ks.drop_duplicates(subset=['key_merge']).reset_index(drop=True)
tr = tr.drop_duplicates(subset=['key_merge']).reset_index(drop=True)

In [587]:
data = exp.dropna(subset=['key_merge']).merge(kr[['amount_money_kr', 'key_merge']].dropna(subset=['key_merge']), on=['key_merge'], how='left').rename(columns={'paid':'TARGET'})

In [588]:
data = data.dropna(subset=['key_merge']).merge(ks[['amount_money_ks', 'key_merge']].dropna(subset=['key_merge']), on=['key_merge'], how='left').rename(columns={'paid':'TARGET'})

In [589]:
data = data.dropna(subset=['key_merge']).merge(tr[['amount_money_tr', 'key_merge']].dropna(subset=['key_merge']), on=['key_merge'], how='left').rename(columns={'paid':'TARGET'})

In [590]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6264 entries, 0 to 6263
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   raw_address       6264 non-null   object        
 1   month             6264 non-null   datetime64[ns]
 2   TARGET            6264 non-null   float64       
 3   geocoder_address  6264 non-null   object        
 4   geocoder_lat      6264 non-null   float64       
 5   geocoder_lon      6264 non-null   float64       
 6   area              6264 non-null   object        
 7   key_merge         6264 non-null   object        
 8   amount_money_kr   499 non-null    float64       
 9   amount_money_ks   355 non-null    float64       
 10  amount_money_tr   1007 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 587.2+ KB


In [591]:
data.shape, weather.shape

((6264, 11), (5423, 10))

In [592]:
weather['key_merge'] = weather['geocoder_address'] + weather['month'].astype(str)

In [593]:
(weather.key_merge.duplicated()).sum()

0

In [594]:
(weather.key_merge.isna()).sum()

0

In [595]:
data = data.merge(weather.drop(columns=['geocoder_address', 'month']), on='key_merge', how='left')

In [596]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6264 entries, 0 to 6263
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   raw_address       6264 non-null   object        
 1   month             6264 non-null   datetime64[ns]
 2   TARGET            6264 non-null   float64       
 3   geocoder_address  6264 non-null   object        
 4   geocoder_lat      6264 non-null   float64       
 5   geocoder_lon      6264 non-null   float64       
 6   area              6264 non-null   object        
 7   key_merge         6264 non-null   object        
 8   amount_money_kr   499 non-null    float64       
 9   amount_money_ks   355 non-null    float64       
 10  amount_money_tr   1007 non-null   float64       
 11  tavg              820 non-null    float64       
 12  tmin              699 non-null    float64       
 13  tmax              761 non-null    float64       
 14  prcp              660 no

In [597]:
weather

,tavg,tmin,tmax,prcp,wspd,pres,tsun,geocoder_address,month,key_merge
0,-2.2,-5.7,1.7,48.1,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-01-01,"Россия, Приморский край, Уссурийск, улица Некр..."
1,3.4,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-02-01,"Россия, Приморский край, Уссурийск, улица Некр..."
2,5.3,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-03-01,"Россия, Приморский край, Уссурийск, улица Некр..."
3,12.7,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-04-01,"Россия, Приморский край, Уссурийск, улица Некр..."
4,15.8,NaN,NaN,NaN,NaN,NaN,NaN,"Россия, Приморский край, Уссурийск, улица Некр...",2016-05-01,"Россия, Приморский край, Уссурийск, улица Некр..."
...,...,...,...,...,...,...,...,...,...,...
5418,20.7,12.9,27.1,62.9,15.3,1011.1,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-06-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5419,23.9,16.9,30.0,112.3,12.7,1007.4,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-07-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5420,22.8,14.6,29.9,11.1,12.8,1012.2,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-08-01,"Россия, Хабаровский край, Верхнебуреинский рай..."
5421,15.4,7.7,22.9,13.2,9.0,1019.7,NaN,"Россия, Хабаровский край, Верхнебуреинский рай...",2023-09-01,"Россия, Хабаровский край, Верхнебуреинский рай..."


In [598]:
data[['amount_money_kr','amount_money_ks','amount_money_tr']] = data[['amount_money_kr','amount_money_ks','amount_money_tr']].fillna(0)

In [599]:
data = data.merge(data_lists.drop_duplicates(subset=['geocoder_address'], keep='first'), on='geocoder_address', how='left')

In [603]:
data=data.drop(columns=['raw_address'])

In [604]:
data.to_feather(r'./data_with_target.frt')

In [606]:
data.columns

Index(['month', 'TARGET', 'geocoder_address', 'geocoder_lat', 'geocoder_lon',
       'area', 'key_merge', 'amount_money_kr', 'amount_money_ks',
       'amount_money_tr', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres',
       'tsun', 'Unnamed: 0', 'Наименование, адрес, реестровый номер объекта',
       'Год постройки', 'Дата принятия на баланс Банка России',
       'Площадь объекта недвижимости \nкв. м', 'Занято службами Банка России',
       'Не занято (не исполь\nзуется)', 'Сдается Банком России в аренду',
       'Аренду\nется Банком России',
       'Передано Банку России по договору безвозмездного пользо-вания',
       'Передано в субаренду', 'Балансовая стоимость, руб.',
       'в том числе переоценка, руб.', 'Сумма начисленной амортизации, руб.',
       'Остаточная стоимость, руб.', 'Техническое состояние',
       'ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ', 'ПУТЬ', 'ВОЗРАСТ ЗДАНИЯ', 'ГОД', 'КВАРТАЛ',
       'clean_address', 'code', 'raw_string'],
      dtype='object')

In [607]:
test_data = pd.read_csv(path + r'/test_data.csv')

In [608]:
test_data

,Наименование вида расходов,Направление расходов,Дата платежа,Отнесено
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,03 2022,NaN
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,04 2022,NaN
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,05 2022,NaN
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,06 2022,NaN
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,07 2022,NaN
...,...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN


In [622]:
def extract_string_for_search(s):
    """
    Извлекает подстроку, которая начинается
    с 'г.' или 'город'
    """
    city = re.search("(г\.|город ).*", s)

    if city is not None:
        start, end = city.span()
        return s[start:end]

    return s

In [623]:
import re

In [624]:
test_data['Направление расходов'].apply(extract_string_for_search)

0       134.1000.01 ДГУ. Оплата за электроэнергию по т...
1       134.1000.01 ДГУ. Оплата за электроэнергию по т...
2       134.1000.01 ДГУ. Оплата за электроэнергию по т...
3       134.1000.01 ДГУ. Оплата за электроэнергию по т...
4       134.1000.01 ДГУ. Оплата за электроэнергию по т...
                              ...                        
4507    134.1000.05 Якутия Оплата потребления газа ул....
4508    134.1000.05 Якутия Оплата потребления газа ул....
4509    134.1000.05 Якутия Оплата потребления газа ул....
4510    134.1000.05 Якутия Оплата потребления газа ул....
4511    134.1000.05 Якутия Оплата потребления газа ул....
Name: Направление расходов, Length: 4512, dtype: object

In [625]:
def add_string_col_to_df(df, raw_string_column="raw_string", result_column="clean_address"):
    """
    Применяет extract_string_for_search к столбцу df[raw_string_column]
    и сохраняет результат в df[result_column]
    """
    df[result_column] = df[raw_string_column].apply(extract_string_for_search)
    return df

In [626]:
add_string_col_to_df(test_data, raw_string_column='Направление расходов')

,Наименование вида расходов,Направление расходов,Дата платежа,Отнесено,clean_address
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,03 2022,NaN,134.1000.01 ДГУ. Оплата за электроэнергию по т...
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,04 2022,NaN,134.1000.01 ДГУ. Оплата за электроэнергию по т...
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,05 2022,NaN,134.1000.01 ДГУ. Оплата за электроэнергию по т...
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,06 2022,NaN,134.1000.01 ДГУ. Оплата за электроэнергию по т...
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,07 2022,NaN,134.1000.01 ДГУ. Оплата за электроэнергию по т...
...,...,...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,134.1000.05 Якутия Оплата потребления газа ул....
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,134.1000.05 Якутия Оплата потребления газа ул....
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,134.1000.05 Якутия Оплата потребления газа ул....
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,134.1000.05 Якутия Оплата потребления газа ул....


In [627]:
geocoder

,geocoder_address,geocoder_lat,geocoder_lon,raw_string
0,None,NaN,NaN,
1,"Россия, Еврейская автономная область, Биробидж...",48.789445,132.928222,"""Город Биробиджа"", город Биробиджан, проспек..."
2,None,NaN,NaN,"6. Наружное освещение сан. зоны санатория ""им...."
3,"Россия, Приморский край, Уссурийск, улица Некр...",43.805100,131.950262,"г. Уссурийск, ул. Некрасова, д. 102"
4,"Россия, Хабаровский край, Советская Гавань, Со...",48.965926,140.281571,"г. Советская Гавань, ул. Советская, 26"
...,...,...,...,...
1770,"Россия, Еврейская автономная область, Биробидж...",48.789445,132.928222,"г Биробиджан, пр-кт 60-летия СССР, д 5"
1771,"Россия, Камчатский край, Петропавловск-Камчатс...",53.068444,158.628735,"г. Петропавловск-Камчатский, ул. Ломоносова,..."
1772,"Россия, Сахалинская область, Южно-Сахалинск, К...",46.958281,142.732004,"г. Южно-Сахалинск, Коммунистический проспект, ..."
1773,"Россия, Чукотский автономный округ, городской ...",64.725052,177.525974,ул колхозная 30б
